In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import datetime

In [ ]:
# Laden der Parquet Datei in einen Dataframe
df = pd.read_parquet('escooter_history.parquet')

In [ ]:
# Übersicht des DF
df.head()

In [ ]:
# Ausweisung der Datentypen der Spalten
df.info()

In [ ]:
# Prüfung nach Null-Werten
df.isna().sum()

In [ ]:
#Prüfen nach Duplikaten
df_dup = df[df.duplicated()]
df_dup

In [ ]:
df["datetime"] = df["datetime"] - datetime.timedelta(2922)

In [ ]:
dup_hourly = df_dup[['holiday']].groupby(df['datetime'].dt.hour).count().rename(columns={'holiday':'count'})
dup_hourly['%'] = dup_hourly['count'] / dup_hourly['count'].sum() * 100
dup_hourly

 Das Ergebniss scheint plausibel zu sein, da die meisten Duplikate während der Rush-Hour vorkommen, deswegen behalten wir die Duplikate im DF

In [ ]:
px.bar(dup_hourly.reset_index(), x = 'datetime', y = '%')

In [ ]:
df['weather_name'] = df['weather']
df['weather'] = df['weather'].replace({'heacy rain or thunderstorm or snow or ice pallets' : 'heavy rain or thunderstorm or snow or ice pallets'})
df['weather'].unique()

In [ ]:
# casting registered_customer as int
df['registered_customer'] = df['registered_customer'].astype(int)
# CATEGORICAL DATA?
df.groupby(df['weather']).count()

In [ ]:
# Erstellung eines dummy DF für die Wetter-Attribute
df_dummy = pd.get_dummies(df[['datetime', 'weather']])
df_dummy['Datum'] = df_dummy.datetime.dt.date
df_dummy = df_dummy.groupby(['Datum']).sum()
df_dummy = df_dummy.reset_index()
df_dummy['Datum'] = pd.to_datetime(df_dummy.Datum)

def get_max_dummy(val):
    st = val.idxmax()
    val[st] = 1
    li = ['weather_clear, few clouds', 'weather_cloudy, mist' ,'weather_heavy rain or thunderstorm or snow or ice pallets','weather_light snow or rain or thunderstorm']
    li.remove(st)
    for el in li:
        val[el] = 0
    return val

df_dummyt = df_dummy.drop('Datum', axis = 1).apply(get_max_dummy, axis=1)
df_dummy = pd.merge(df_dummy[['Datum']].reset_index(), df_dummyt.reset_index(), on='index')
df_dummy = df_dummy.drop('index', axis = 1)
df_dummy = df_dummy.set_index('Datum')
df_dummy

In [ ]:
# encode weather using map
### SCHREIBFEHLER verbessern
df['weather_name'] = df['weather'].map({'clear, few clouds':0,'cloudy, mist':1,'light snow or rain or thunderstorm':2,'heavy rain or thunderstorm or snow or ice pallets':3})
df['weather_name']

In [ ]:
df_count_hourly = df[['datetime','registered_customer']].resample('1H',on='datetime').agg({'datetime':'count','registered_customer':'sum'}).rename(columns={'datetime':'Bookings'})
df_count_hourly.head()
df_count_hourly['unregistered_customer'] = df_count_hourly['Bookings'] - df_count_hourly['registered_customer']

In [ ]:
df_mean_hourly = df.resample('1H',on='datetime').mean()
df_mean_hourly.drop(['registered_customer'],axis=1,inplace=True)

In [ ]:
df_hourly = pd.merge(df_count_hourly,df_mean_hourly,on='datetime').reset_index()
df_hourly.head()

In [ ]:
#### löschen von unnötigen spalten
df_hourly['day_of_week'] = df_hourly['datetime'].dt.dayofweek
df_hourly['weekday'] = df_hourly['datetime'].dt.day_name()
season_dict = { 1 : "Winter", 2 : "Spring", 3 : "Summer", 4 :"Autumn"}
df_hourly["season"] = df_hourly["datetime"].dt.month % 12 // 3 + 1
df_hourly["season"] =  df_hourly["season"].map(season_dict)
df_hourly['week_of_year'] =  df_hourly['datetime'].dt.isocalendar().week
df_hourly['hour'] = df_hourly['datetime'].dt.hour
df_hourly['day'] = df_hourly['datetime'].dt.day
df_hourly['month'] = df_hourly['datetime'].dt.month
df_hourly['year'] =  df_hourly['datetime'].dt.year
df_hourly['date'] = df_hourly['datetime'].dt.date

#df_hourly['weekday'] = df['weekday'].map({''})

In [ ]:
df_hourly.head()

In [ ]:
df_hourly['day_of_week'].unique()

In [ ]:
df_hourly.describe().T

In [ ]:
# drop columns where no bookings happend
df_hourly = df_hourly[~(df_hourly['Bookings']==0)]
df_hourly
#df_hourly.fillna(method='ffill',inplace=True)

In [ ]:
#drop datetime not needed right now()
df_hourly = df_hourly.drop(['datetime'],axis=1)

In [ ]:
df_hourly.info()

### DataFrame erstellen basierend auf täglichen Werten

In [ ]:
df_day = df.set_index('datetime')
df_day.index = pd.to_datetime(df_day.index)
df_day_mean = df_day.resample('d').mean()
df_day_mean = df_day_mean.drop('registered_customer', axis = 1)
df_day_bookings = df_day.resample('d')[['temp', 'registered_customer']].agg({'temp' : 'count', 'registered_customer' : 'sum'})
df_day_bookings.columns = ['Bookings', 'registered_customer']
df_day = pd.merge(df_day_mean.reset_index(), df_day_bookings.reset_index(), on = 'datetime')
df_day['weather'] = df_day['weather'].astype('int')
df_day['unregistered_customer'] = df_day.Bookings - df_day.registered_customer
df_day

In [ ]:
# Hinzufügen der Dummy Wetter Attribute
df_day = pd.merge(df_day, df_dummy.reset_index(), left_on='datetime', right_on='Datum')

In [ ]:
# Zeitwerte werden hinzugefügt
df_day['day_of_week'] = df_day['datetime'].dt.dayofweek
df_day['season'] =  df_day['datetime'].dt.month%12 // 3 + 1
df_day['week_of_year'] =  df_day['datetime'].dt.isocalendar().week
df_day['day'] = df_day['datetime'].dt.day
df_day['month'] = df_day['datetime'].dt.month
df_day['year'] =  df_day['datetime'].dt.year
df_day

In [ ]:
# Beschriftung der Abbildung mit median min max

## Buchungen 2011 vs 2012

In [ ]:
df_daily_19 = df_day[df_day['year'] == 2011]
df_daily_20 = df_day[df_day['year'] == 2012]
df_daily_19.head()

In [ ]:
df_m = df_day.copy()
df_m['quarter'] =df_m['datetime'].dt.quarter
df_tt = df_m.groupby(['year','quarter'])[['Bookings']].sum()
df_tt=df_tt.iloc[::,:1]
df_tt = df_tt.reset_index()
df_tt.columns = ['year', 'quarter', 'Bookings']
# df_tt['vorquartal'] = df_tt.Bookings.shift()
# df_tt['wachstum'] = df_tt['Bookings']/df_tt['vorquartal']
# df_tt_2011 =df_tt[df_tt['year']==2011]
# df_tt_2012 =df_tt[df_tt['year']==2012]
df_tt_2011 =df_tt[df_tt['year']==2011]
df_tt_2011 = df_tt_2011[['quarter', 'Bookings']]
df_tt_2011.columns = ['quarter', 'Bookings_2011']
df_ttt = pd.merge(df_tt, df_tt_2011, on = 'quarter')
# df_ttt = df_tt_2011.sort_values()
df_tt_2012 =df_ttt[df_ttt['year']==2012]
df_tt_2012['Änderung'] = df_ttt['Bookings'] / df_ttt['Bookings_2011']
px.bar(df_tt_2012, x = 'quarter', y = 'Änderung')

In [ ]:
df_m = df_day.copy()
df_m['quarter'] =df_m['datetime'].dt.quarter
df_tt = df_m.groupby(['year'])[['Bookings']].sum()
df_tt = df_tt.reset_index()
df_tt['Vorjahr'] = df_tt.Bookings.shift()
df_tt['Änderung'] = df_tt['Bookings'] / df_tt['Vorjahr']
print(f"Die Änderungsrate zwischen den Jahren beträgt : {df_tt.iloc[1]['Änderung']}")

In [ ]:
px.bar(df_tt, x = 'quarter',y = 'wachstum', color = 'year')

In [ ]:
df_month_19 = df_daily_19[['month','Bookings']].groupby('month').sum()
df_month_20 = df_daily_20[['month','Bookings']].groupby('month').sum()

In [ ]:
# Umbennenung Legende und Y-Achse

In [ ]:
fig = px.line(df_month_19.reset_index(), x='month', y=['Bookings',df_month_20['Bookings']], title = 'Vergleich der Buchungen 2011 und 2012 auf Monatsbasis')
fig.show()

## Histogram Count -> skewed towards right -> Normalisation?

In [ ]:
# px histonorm = 'probability density' 
# Aussage Diagramm ?

In [ ]:
plt.figure(figsize=(16,8))
sns.displot(df_hourly['Bookings'], kde = True)
plt.show()
# Normalisieren

## Analyse der Wetterdaten

In [ ]:
df_year = df.resample('1Y', on='datetime')[['temp','atemp','humidity','windspeed']].agg(['mean','min','max'])
df_year.T

In [ ]:

fig = plt.figure(figsize=[12,10])
ax1 = fig.add_subplot(2,2,1)
ax1 = sns.displot(df.temp,bins=range(int(df.temp.min()),int(df.temp.max())),kde=True)


ax2 = fig.add_subplot(2,2,2)
ax2 = sns.displot(df.atemp,bins=range(int(df.atemp.min()),int(df.atemp.max())),kde=True)


ax3 = fig.add_subplot(2,2,3)
ax3 = sns.displot(df.humidity,bins=range(int(df.humidity.min()),int(df.humidity.max())),kde=True)


ax4 = fig.add_subplot(2,2,4)
ax4 = sns.displot(df.windspeed,bins=range(int(df.windspeed.min()),int(df.windspeed.max())),kde=True)

In [ ]:
df_count = df_hourly['Bookings'].groupby(df_hourly['weather']).mean()
df_count

In [ ]:
# Verteilung der Buchung über die einzelnen Wetterkategorien

df_weather_cat_sum = df[['datetime','weather']].groupby('weather')['datetime'].count().reset_index(name='Bookings')
df_weather_cat_sum['%'] = df_weather_cat_sum['Bookings'] / len(df) * 100
df_weather_cat_sum['weather_name'] = df_weather_cat_sum['weather'].map({0:'clear, few clouds',1:'cloudy, mist',2:'light snow or rain or thunderstorm',3:'heavy rain or thunderstorm or snow or ice pallets'})
df_weather_cat_sum

In [ ]:
df_day.head()

In [ ]:
# Nur 3 Stunden im Jahr bei denen es stark regnet oder stürmt
df_w_2 = df_hourly[df_hourly['weather'] == 3]
df_w_2

In [ ]:
fig = plt.figure(figsize=[12,10])
ax1 = fig.add_subplot(2,1,1)
ax1 = sns.pointplot(x='month',y='Bookings',hue='weather',data=df_hourly.groupby(['weather','month'])['Bookings'].mean().reset_index())

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(ncols=4)
fig.set_size_inches(12, 5)
sns.regplot(x="temp", y="Bookings", data=df_day,ax=ax1)
sns.regplot(x="atemp", y="Bookings", data=df_day,ax=ax2)
sns.regplot(x="windspeed", y="Bookings", data=df_day,ax=ax3)
sns.regplot(x="humidity", y="Bookings", data=df_day,ax=ax4)

In [ ]:
df_temp = df_hourly.copy()
df_temp.reset_index(inplace=True)
df_temp['bin_temp'] = pd.cut(df_temp['temp'], [0, 5,10,15,20,25,30,35,40,45])
plt.figure(figsize=(8,4))
ax = sns.barplot(x="bin_temp", y="Bookings", data=df_temp,
                 palette="flare")
ax.set(xlabel='Temperatur in °C', ylabel='Anzahl der Buchungen', title='Übersicht Anzahl der Buchungen je Temperatur')

In [ ]:
df_temp[df_temp['temp'] >= 40]

### Wetterdatenauswertung auf die einzelnen Jahre bezogen

# Registrierte vs nicht registrierte User

In [ ]:
# Customers overall
sum_of_cust = df['registered_customer'].count()
sum_of_cust

In [ ]:
# Overview
df.registered_customer.value_counts()

In [ ]:
# Number of registered customers
number_reg_cust = df.registered_customer[df.registered_customer==True].count()
number_reg_cust

In [ ]:
# Number of not registered customers
number_N_reg_cust = df.registered_customer[df.registered_customer==False].count()
number_N_reg_cust

In [ ]:
# Ratio registered to all customers
ratio_reg_cust = number_reg_cust / sum_of_cust
ratio_reg_cust

In [ ]:

# Ratio not registered to all customers
ratio_N_reg_cust = number_N_reg_cust / sum_of_cust
ratio_N_reg_cust

In [ ]:
# daily overview registered_customer
df_registered = df_day[['Bookings', 'registered_customer']]
df_registered

In [ ]:
# calculation registration_rate per day
df_customers_d = df_registered.copy()
df_customers_d['unregistered_customers'] = df_day['Bookings']-df_day['registered_customer']
df_customers_d['registration_rate'] = df_day['registered_customer'] / df_customers_d['Bookings']
df_customers_d

In [ ]:
# monthly overview registered_customer
df_month = df_day.groupby(['year','month'])[['Bookings', 'registered_customer']].agg({'Bookings':'sum','registered_customer':'sum'}).reset_index()
df_month.sort_values(['year', 'month'], ascending=True)

In [ ]:
# calculation registration_rate per month
df_customers_m = df_month.copy()
df_customers_m['unregistered_customers'] = df_month['Bookings']-df_month['registered_customer']
df_customers_m['registration_rate'] = df_month['registered_customer'] / df_customers_m['Bookings']
df_customers_m

In [ ]:
# quarter overview registered_customer
df_season = df_day.groupby(['year','season'])[['Bookings', 'registered_customer']].agg({'Bookings':'sum','registered_customer':'sum'}).reset_index()
df_season.sort_values(['year', 'season'], ascending=True)

In [ ]:
df_customers_q = df_season.copy()
df_customers_q['unregistered_customers'] = df_season['Bookings']-df_season['registered_customer']
df_customers_q['registration_rate'] = df_season['registered_customer'] / df_customers_q['Bookings']
df_customers_q

In [ ]:
# year overview registered_customer
df_year = df_day.groupby(['year'])[['Bookings', 'registered_customer']].agg({'Bookings':'sum','registered_customer':'sum'}).reset_index()
df_year

In [ ]:
df_customers_y = df_year.copy()
df_customers_y['unregistered_customers'] = df_year['Bookings']-df_year['registered_customer']
df_customers_y['registration_rate'] = df_year['registered_customer'] / df_customers_y['Bookings']
df_customers_y

In [ ]:
df_holiday = df_day.groupby(['year', 'holiday'])[['Bookings', 'registered_customer']].agg({'Bookings':'sum','registered_customer':'sum'}).reset_index()
df_holiday

In [ ]:
# calculation registration_rate holiday
df_customers_h = df_holiday.copy()
df_customers_h['unregistered_customers'] = df_holiday['Bookings']-df_holiday['registered_customer']
df_customers_h['registration_rate'] = df_holiday['registered_customer'] / df_customers_h['Bookings']
df_customers_h

In [ ]:
df_workingday = df_day.groupby(['year', 'workingday'])[['Bookings', 'registered_customer']].agg({'Bookings':'sum','registered_customer':'sum'}).reset_index()
df_workingday

In [ ]:
# calculation registration_rate workingday
df_customers_w = df_workingday.copy()
df_customers_w['unregistered_customers'] = df_workingday['Bookings']-df_workingday['registered_customer']
df_customers_w['registration_rate'] = df_workingday['registered_customer'] / df_customers_w['Bookings']
df_customers_w

In [ ]:
df

In [ ]:
df['Jahr'] = df.datetime.dt.year

In [ ]:
pd.crosstab(df['Jahr'], df['registered_customer'], margins=True, normalize=True)

In [ ]:
sns.countplot(data=df, x='registered_customer')

In [ ]:
df['tag'] = df.datetime.dt.day_name()
px.histogram(df, x='tag', histnorm='probability density', color='registered_customer')

In [ ]:
df_hourly.groupby(['workingday', 'hour'])[['Bookings']].sum().reset_index()

In [ ]:
df_hourly

In [ ]:
df_hourly.groupby(['day_of_week', 'hour'])[['Bookings']].mean().reset_index()

In [ ]:
df_hourly.groupby(['day_of_week', 'hour'])[['Bookings']].mean().reset_index()

## Analysen

In [ ]:
# Plots of average count across hour in a day for various categories

f, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 18))
group_work_hour = df_hourly.groupby(['workingday', 'hour'])[['Bookings']].mean().reset_index()
sns.pointplot(data=group_work_hour, x='hour', y='Bookings', hue='workingday', ax=axes[0], legend=True)
handles, _ = axes[0].get_legend_handles_labels()
axes[0].legend(handles, ['Kein Arbeitstag', 'Arbeitstag'])
axes[0].set(xlabel='Stunde des Tages', ylabel='Durchschnittliche Bookings', title='Durchschnittliche Bookings pro Stunde abhängig vom Arbeitstag')

hue_order= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
group_day_hour = df_hourly.groupby(['weekday', 'hour'])[['Bookings']].mean().reset_index()
sns.pointplot(data=group_day_hour, x='hour', y='Bookings', hue='weekday', ax=axes[1], hue_order=hue_order)
axes[1].set(xlabel='Stunde des Tages', ylabel='Durchschnittliche Bookings', title='Durchschnittliche Bookings pro Stunde abhängig vom Wochentag')


plt.show()

In [ ]:
group_work_hour

In [ ]:
df_viz_h = df.groupby(['hour', 'registered_customer'], as_index=False)[['temp']].count()
df_viz_h.columns = ['hour', 'registered_customer', 'Bookings']
df_viz_h.head()

In [ ]:
# Daily driving profile registered customers vs. not registered customers
fig_h = px.line(df_viz_h, x='hour', y='Bookings', color='registered_customer', title=" Nutzung registrierte vs. nicht registrierte Kunden auf Stundenbasis")
fig_h.show()

In [ ]:
df['kalenderwoche'] = df.datetime.dt.isocalendar().week
df_viz_cw2 = df.groupby(['Jahr','kalenderwoche', 'registered_customer'], as_index=False)[['temp']].count()
df_viz_cw2.columns = ['Jahr','kalenderwoche', 'registered_customer', 'Bookings']
df_viz_cw2.head()

In [ ]:
df_viz_cw2['cw_year'] = df_viz_cw2['kalenderwoche'].astype(str) + ' / ' + df_viz_cw2['Jahr'].astype(str)
df_viz_cw2 = df_viz_cw2.sort_values(['Jahr','kalenderwoche'], ascending=True).reset_index(drop=True)
df_viz_cw2.head()

In [ ]:
fig_cw2 = px.line(df_viz_cw2, x='cw_year',y='Bookings', color='registered_customer', title=" Nutzung registrierte vs. nicht registrierte Kunden pro Kalenderwoche")
fig_cw2.show()
# zu sehen, dass inbesondere bei den Ferientagen ein Einbruch registrierter Kunden stattfindet

In [ ]:
# workaround um Wochentage in richtige Reihenfolge zu haben
sorter = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sorterIndex = dict(zip(sorter,range(len(sorter))))
sorterIndex

In [ ]:
df['day_id'] = df.index
df['day_id'] = df['tag'].map(sorterIndex)
df

In [ ]:
# Anzeige nach Wochentag über Season, Betrachtung saisonaler Einflüsse
season_dict = { 1 : "Winter", 2 : "Spring", 3 : "Summer", 4 :"Autumn"}
df["season"] = df["datetime"].dt.month % 12 // 3 + 1
df["season"] =  df["season"].map(season_dict)
df_viz_day_season = df.groupby(['season','tag', 'day_id', 'registered_customer'], as_index=False)[['temp']].count()
df_viz_day_season.columns = ['season','tag', 'day_id', 'registered_customer', 'Bookings']
df_viz_day_season.head()

In [ ]:
df_viz_day_season['day_season'] = df_viz_day_season['tag'].astype(str) + ' / ' + df_viz_day_season['season'].astype(str)
df_viz_day_season = df_viz_day_season.sort_values(['season','day_id'], ascending=True).reset_index(drop=True)
df_viz_day_season.head()

In [ ]:
fig_day_season = px.line(df_viz_day_season, x='day_season',y='Bookings', color='registered_customer', title=" Nutzung registrierte vs. nicht registrierte Kunden pro Wochentag je Jahreszeit")
fig_day_season.show()
# bei registrierten Nutzern sind im Winter und Frühling insbesondere Mittwochs stärkere Rückgänge zu sehen

## Analyse der bedingten Häufigkeiten
# Analyse mittels dem chi2 Test


In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2


### Abhängigkeit zwischen Jahr und Registered_customer


In [ ]:
df_k = df.copy()
df_k['Hour'] = df_k.datetime.dt.hour
df_k['Jahr'] = df_k.datetime.dt.year
table = pd.crosstab(df_k['registered_customer'], df_k['Jahr'])
table


In [ ]:
rel = pd.crosstab(df_k['registered_customer'], df_k['Jahr'], margins=True, normalize=True).round(2)
rel


In [ ]:
ch2, p, dof, expected = chi2_contingency(table)
# Interpretation des Ergebnisses
alpha = 0.05 # Standard level for "significant"
prob = 1-alpha
critical = chi2.ppf(prob, dof) # liefert das <prob>-Quantil der chi2-Verteilung mit <dof> Freiheitsgraden
print(f'Bei einer Signifikanz von alpha={alpha}, ist die Testentscheidung:')
if ch2 >= critical:
	print(f'Abhängig (H0 kann wiederlegt werden)')
else:
	print('Unabhängig (fail to reject H0)')


In [ ]:
# Jahr 2011 oder 2012
# Registered_customer : 1 = Benutzer ist registiert
#                       0 = Benutzer ist nicht registriert
P2019_1 = rel.reset_index().iloc[1][2011]
P2019 = rel.reset_index().iloc[2][2011]
P1bed2019 =  P2019_1 / P2019
P2020_1 = rel.reset_index().iloc[1][2012]
P2020 = rel.reset_index().iloc[2][2012]
P1bed2020 =  P2020_1 / P2020
print(f'Wahrscheinlichkeit für P(1|2011) = {P1bed2019}')
print(f'Wahrscheinlichkeit für P(1|2012) = {P1bed2020}')


Da die bedingte Wahrscheinlichkeit bei einem Zufallsexperiment eine registrierte Person zu ziehen ist abhängig von dem Jahr. Unter der Bedingung, dass das Jahr bekannt ist verändert sich die Wahrscheinlichkeit. Außerdem kann hier erkannt werden, dass die Wahrscheinlichkeit für eine registrierte Person 2012 höher ist als 2011. 


### Abhängigkeit zwischen registered_customer und weather


In [ ]:
table = pd.crosstab(df_k['registered_customer'], df_k['weather'])
table


In [ ]:
rel = pd.crosstab(df_k['registered_customer'], df_k['weather'], margins=True, normalize=True)
rel


In [ ]:
ch2, p, dof, expected = chi2_contingency(table)
# Interpretation des Ergebnisses
alpha = 0.05 # Standard level for "significant"
prob = 1-alpha
critical = chi2.ppf(prob, dof) # liefert das <prob>-Quantil der chi2-Verteilung mit <dof> Freiheitsgraden
print(f'Bei einer Signifikanz von alpha={alpha}, ist die Testentscheidung:')
if ch2 >= critical:
	print(f'Abhängig (H0 kann wiederlegt werden)')
else:
	print('Unabhängig (fail to reject H0)')


In [ ]:
# Weather : 1=clear, 2=cloudy, 3=snow, 4=heavy_rain
# Registered_customer : 1 = Benutzer ist registiert
#                       0 = Benutzer ist nicht registriert
Pclear_1 = rel.reset_index().iloc[1][0]
Pclear = rel.reset_index().iloc[2][0]
P1bedclear =  Pclear_1 / Pclear
Pcloudy_1 = rel.reset_index().iloc[1][1]
Pcloudy = rel.reset_index().iloc[2][1]
P1bedcloudy =  Pcloudy_1 / Pcloudy
Pheavy_1 = rel.reset_index().iloc[1][3]
Pheavy = rel.reset_index().iloc[2][3]
P1bedheavy =  Pheavy_1 / Pheavy
Psnow_1 = rel.reset_index().iloc[1][2]
Psnow = rel.reset_index().iloc[2][2]
P1bedsnow =  Psnow_1 / Psnow

print(f'Wahrscheinlichkeit für P(1|clear) = {P1bedclear}')
print(f'Wahrscheinlichkeit für P(1|cloudy) = {P1bedcloudy}')
print(f'Wahrscheinlichkeit für P(1|heavy) = {P1bedheavy}')
print(f'Wahrscheinlichkeit für P(1|snow) = {P1bedsnow}')


Hier kann erkannt werden, dass gerade bei schlechtem Wetter die Wahrscheinlichkeit für eine registrierte Person steigt, beispielsweise sind bei der Wetterkategorie 'heavy rain or thunderstorm or snow or ice pallets' 96.25% der Kunden registriert.


### Abhängigkeit zwischen registered_customer und workingday


In [ ]:
table = pd.crosstab(df_k['registered_customer'], df_k['workingday'])
table


In [ ]:
rel = pd.crosstab(df_k['registered_customer'], df_k['workingday'], margins=True, normalize=True)
rel


In [ ]:
ch2, p, dof, expected = chi2_contingency(table)
# Interpretation des Ergebnisses
alpha = 0.05 # Standard level for "significant"
prob = 1-alpha
critical = chi2.ppf(prob, dof) # liefert das <prob>-Quantil der chi2-Verteilung mit <dof> Freiheitsgraden
print(f'Bei einer Signifikanz von alpha={alpha}, ist die Testentscheidung:')
if ch2 >= critical:
	print(f'Abhängig (H0 kann wiederlegt werden)')
else:
	print('Unabhängig (H0 kann nicht wiederlegt werden)')


In [ ]:
# Workingday : y / n
# Registered_customer : 1 = Benutzer ist registiert
#                       0 = Benutzer ist nicht registriert
Pn_1 = rel.reset_index().iloc[1][0]
Pn = rel.reset_index().iloc[2][0]
P1bedn =  Pn_1 / Pn
Py_1 = rel.reset_index().iloc[1][1]
Py = rel.reset_index().iloc[2][1]
P1bedy =  Py_1 / Py


print(f'Wahrscheinlichkeit für P(1|no) = {P1bedn}')
print(f'Wahrscheinlichkeit für P(1|yes) = {P1bedy}')


Unter der Bedingung, dass es kein Arbeitstag ist, sinkt auch die Wahrscheinlichkeit für die registrierten Kunden, da nurnoch 68.7% der Kunden registriert sind. Prüft man wiederum in einem Zufallsexperiment an einem Arbeitstag ob es sich um einen registrierten Kunden handelt, so steigt die Wahrscheinlichkeit, das es sich um einen Registrierten Kunden handelt auf 86.9%. 


In [ ]:
### Abhängigkeit zwischen registered_customer und holiday
table = pd.crosstab(df_k['registered_customer'], df_k['holiday'])
table
rel = pd.crosstab(df_k['registered_customer'], df_k['holiday'], margins=True, normalize=True)
rel
ch2, p, dof, expected = chi2_contingency(table)
# Interpretation des Ergebnisses
alpha = 0.05 # Standard level for "significant"
prob = 1-alpha
critical = chi2.ppf(prob, dof) # liefert das <prob>-Quantil der chi2-Verteilung mit <dof> Freiheitsgraden
print(f'Bei einer Signifikanz von alpha={alpha}, ist die Testentscheidung:')
if ch2 >= critical:
	print(f'Abhängig (H0 kann wiederlegt werden)')
else:
	print('Unabhängig (H0 kann nicht wiederlegt werden)')


In [ ]:
# Holiday : y / n
# Registered_customer : 1 = Benutzer ist registiert
#                       0 = Benutzer ist nicht registriert
Pn_1 = rel.reset_index().iloc[1][0]
Pn = rel.reset_index().iloc[2][0]
P1bedn =  Pn_1 / Pn
Py_1 = rel.reset_index().iloc[1][1]
Py = rel.reset_index().iloc[2][1]
P1bedy =  Py_1 / Py


print(f'Wahrscheinlichkeit für P(1|no) = {P1bedn}')
print(f'Wahrscheinlichkeit für P(1|yes) = {P1bedy}')


Die Wahrscheinlichkeit bei einem Zufallexperiment für eine Person welche registriert ist, ist an einem Tag an dem keine Ferien ist 81.47% und damit größer als an einem Tag an dem Ferien sind, da hier die Wahrscheinlichkeit nur 74.5% beträgt.


### Abhängigkeit zwischen workingday und weather 


In [ ]:
table = pd.crosstab(df_k['workingday'], df_k['weather'])
table


In [ ]:
rel = pd.crosstab(df_k['workingday'], df_k['weather'], margins=True, normalize=True)
rel


In [ ]:
ch2, p, dof, expected = chi2_contingency(table)
# Interpretation des Ergebnisses
alpha = 0.05 # Standard level for "significant"
prob = 1-alpha
critical = chi2.ppf(prob, dof) # liefert das <prob>-Quantil der chi2-Verteilung mit <dof> Freiheitsgraden
print(f'Bei einer Signifikanz von alpha={alpha}, ist die Testentscheidung:')
if ch2 >= critical:
	print(f'Abhängig (H0 kann wiederlegt werden)')
else:
	print('Unabhängig (H0 kann nicht wiederlegt werden)')


In [ ]:
# Weather : clear, cloudy, heavy_rain, snow
# Workingday :          1 = Tag ist ein Arbeitstag
#                       0 = Tag ist kein Arbeitstag
Pclear_1 = rel.reset_index().iloc[1][0]
Pclear = rel.reset_index().iloc[2][0]
P1bedclear =  Pclear_1 / Pclear
Pcloudy_1 = rel.reset_index().iloc[1][1]
Pcloudy = rel.reset_index().iloc[2][1]
P1bedcloudy =  Pcloudy_1 / Pcloudy
Pheavy_1 = rel.reset_index().iloc[1][3]
Pheavy = rel.reset_index().iloc[2][3]
P1bedheavy =  Pheavy_1 / Pheavy
Psnow_1 = rel.reset_index().iloc[1][2]
Psnow = rel.reset_index().iloc[2][2]
P1bedsnow =  Psnow_1 / Psnow

print(f'Wahrscheinlichkeit für P(1|clear) = {P1bedclear}')
print(f'Wahrscheinlichkeit für P(1|cloudy) = {P1bedcloudy}')
print(f'Wahrscheinlichkeit für P(1|heavy) = {P1bedheavy}')
print(f'Wahrscheinlichkeit für P(1|snow) = {P1bedsnow}')


Bei diesen bedingten Häufigkeiten muss aufgepasst werden, dass nicht von einer Kausalen Abhängigkeit ausgegangen werden kann, da aus logischer Sicht keine Beeinflussung des Wetters und des Arbeitstags vorliegt. Der Zusammehang geht zwar aus den Zahlen hervor, allerdings ist der Wirkungszusammenhang in der Realität eher fragwürdig. 


### Abhängigkeit zwischen workingday und weather unter der Bedingung das registered_customer bekannt ist


In [ ]:
mask = df_k.registered_customer == 1
df_km = df_k[mask]
table = pd.crosstab(df_km['workingday'], df_km['weather'])
table


In [ ]:
rel = pd.crosstab(df_km['workingday'], df_km['weather'], margins=True, normalize=True)
rel


In [ ]:
ch2, p, dof, expected = chi2_contingency(table)
# Interpretation des Ergebnisses
alpha = 0.05 # Standard level for "significant"
prob = 1-alpha
critical = chi2.ppf(prob, dof) # liefert das <prob>-Quantil der chi2-Verteilung mit <dof> Freiheitsgraden
print(f'Bei einer Signifikanz von alpha={alpha}, ist die Testentscheidung:')
if ch2 >= critical:
	print(f'Abhängig (H0 kann wiederlegt werden)')
else:
	print('Unabhängig (H0 kann nicht wiederlegt werden)')


In [ ]:
# Weather : clear, cloudy, heavy_rain, snow
# Workingday :          1 = Tag ist ein Arbeitstag
#                       0 = Tag ist kein Arbeitstag
Pclear_1 = rel.reset_index().iloc[1][0]
Pclear = rel.reset_index().iloc[2][0]
P1bedclear =  Pclear_1 / Pclear
Pcloudy_1 = rel.reset_index().iloc[1][1]
Pcloudy = rel.reset_index().iloc[2][1]
P1bedcloudy =  Pcloudy_1 / Pcloudy
Pheavy_1 = rel.reset_index().iloc[1][3]
Pheavy = rel.reset_index().iloc[2][3]
P1bedheavy =  Pheavy_1 / Pheavy
Psnow_1 = rel.reset_index().iloc[1][2]
Psnow = rel.reset_index().iloc[2][2]
P1bedsnow =  Psnow_1 / Psnow

print(f'Wahrscheinlichkeit für P(1|clear) = {P1bedclear}')
print(f'Wahrscheinlichkeit für P(1|cloudy) = {P1bedcloudy}')
print(f'Wahrscheinlichkeit für P(1|heavy) = {P1bedheavy}')
print(f'Wahrscheinlichkeit für P(1|snow) = {P1bedsnow}')


Somit wurde auf folgendes Modell geprüft:
weahter -> registrierten_kunden <- workingday
Da allerdings weather und workingday im Datensatz eine abhängigkeit haben unabhängig ob die Kunden registriert sind oder nicht muss das Modell wiederlegt werden. 

# Welchen Einfluss hat das Wetter

# Nutzungsverhalten in Zeit Dimension

In [ ]:
fig_overall_count = px.line(df_day.reset_index(),y='Bookings', x="datetime", title = 'Anzahl der Buchungen 2011-2012')
fig_overall_count.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")])))
fig_overall_count.show()

In [ ]:

df_VizSeasons=pd.DataFrame(df_hourly.groupby('season').sum()['Bookings'].sort_values(ascending=False)).reset_index()
df_VizSeasons.style.background_gradient(cmap=sns.light_palette("green", as_cmap=True))

In [ ]:
df_by_season = df_hourly.groupby("season")["Bookings"].sum().reset_index(name="rented_bikes")
fig_pie = px.pie(df_by_season,values="rented_bikes",names="season", title="Anzahl der ausgeliehenen E-Scooter pro Jahreszeit")
fig_pie.show()

In [ ]:
hourly_sum = df_hourly.groupby(df_hourly["hour"]).agg({"Bookings": "sum"})
fig = px.bar(hourly_sum, title="Stündliche Verteilung der Buchungen innerhalb 2 Jahren", labels= {"value" : "Gesamtanzahl der Buchungen pro Stunde", "datetime": "Uhrzeit in Stunde"})
fig.update_layout(showlegend=False)

In [ ]:

df_avg_per_day_all = df.groupby([df["datetime"].dt.isocalendar().year,df["datetime"].dt.isocalendar().week,df["datetime"].dt.isocalendar().day])["registered_customer"].sum().reset_index(name="sum_of_rentals_per_day")
df_avg_per_day_all  = df_avg_per_day_all.groupby(["year","week"])["sum_of_rentals_per_day"].mean().reset_index(name="avg_per_wk")
px.line(df_avg_per_day_all,x="week",y="avg_per_wk",color="year",markers=True, title='Durchschnittliche Buchungen pro Woche für 2011 und 2012',
        labels = {'avg_per_wk' : 'Durchschnitt pro Woche'})

In [ ]:
df_avg_per_day_all.groupby("year").mean()

In [ ]:
df_time_index = df.set_index('datetime')
df_viz = df_time_index.groupby([pd.Grouper(freq='1W')]).agg({'registered_customer':'count'}).reset_index()
fig = px.line(df_viz,x="datetime",y="registered_customer",color=df_viz["datetime"].dt.year)
# fig.update_layout(legend = {'2011' : '2011', '2012' : '2012'}) 
# legende Bearbeiten!
fig.show()

In [ ]:
df_week = df_day[(df_day["day_of_week"] != 5) & (df_day["day_of_week"] != 6)]
df_viz_pW = df_week.groupby("day_of_week")["Bookings"].sum()
px.bar(df_viz_pW)

In [ ]:
df_weekend = df_day[~((df_day["day_of_week"] != 5) & (df_day["day_of_week"] != 6))]
df_viz_WeEnd = df_weekend.groupby("day_of_week")["Bookings"].sum()
px.bar(df_viz_WeEnd)

In [ ]:
df_vizT = df_hourly.groupby(["temp"])[["Bookings"]].sum().reset_index()
figT = px.histogram(df_vizT, x="temp", y="Bookings", nbins=15 )
figT.update_layout(showlegend=False,bargap=0.25)

In [ ]:
df_vizAt = df_hourly.groupby(["atemp"])[["Bookings"]].sum().reset_index()
figAt = px.histogram(df_vizAt, x="atemp", y="Bookings", nbins=15 )
figAt.update_layout(showlegend=False,bargap=0.25)

# Korrelationsmatrix

In [ ]:
corrMatt = df_day[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'registered_customer', 'unregistered_customer','Bookings', 'year',
                   'season', 'weather', 'week_of_year']].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
# fig,ax= plt.subplots()
# fig.set_size_inches(20,10)
sns.set(rc = {'figure.figsize' : (20,10)})
sns.heatmap(corrMatt, annot=True, cmap='coolwarm', mask = mask).set(title='Korrelationsmatrix über ausgwählte Features')

# Prognose
## Zeitreihenanalyse

### Preprocessing

In [ ]:
# Resample des DF auf den Monat
df_time = df.set_index('datetime') 
df_time.index = pd.to_datetime(df_time.index) # Sichergehen, dass Index das richtige Format hat. 
df_time = df_time.resample('M').count()[['temp']]
df_time.columns = ['Bookings']
df_time['Vorheriger_Monat'] = df_time['Bookings'].shift()

# Einfügen von Mulitplikativen und Additiven Unterschieden
df_time['Mul'] = df_time['Bookings'] / df_time['Vorheriger_Monat']
df_time['Add'] = df_time['Bookings'] - df_time['Vorheriger_Monat']

# Einfügen des Rolling Average
df_time['Rol_avg'] = df_time['Bookings'].rolling(window = 3).mean().round(2)
df_time['Monat_t'] = range(1,25)
df_time

Eine weitere Möglichkeit zur Analyse der Daten in Bezug auf die Bookings ist das Betrachten als Zeitreihe. Hier werden die Bookings über die 2 Jahre hinweg Beobachtet und die Entwicklung der Größe analysiert. Die Folgende Grafik zeigt die Zeitliche Entwicklung der Verkäufe, aggregiert auf den Monat und die gleitenden Durchschnitte der Zeitreihe der Ordnung drei. 

In [ ]:
fig = px.line(df_time.reset_index(),x = 'datetime', y = ['Bookings', 'Rol_avg'])
fig.show()

In [ ]:
print(f"Max von 2011 : {df_time.loc['2011']['Bookings'].idxmax()} mit {df_time.loc['2011']['Bookings'].max()} Bookings")
print(f"Min von 2011 : {df_time.loc['2011']['Bookings'].idxmin()} mit {df_time.loc['2011']['Bookings'].min()} Bookings")
print(f"Max von 2012 : {df_time.loc['2012']['Bookings'].idxmax()} mit {df_time.loc['2012']['Bookings'].max()} Bookings")
print(f"Min von 2012 : {df_time.loc['2012']['Bookings'].idxmin()} mit {df_time.loc['2012']['Bookings'].min()} Bookings")

Auf beiden Kurven kann ein positiver Trend gesehen werden, jedoch können auch Saisonale Einflüsse erkannt werden, welche je nach Monat die Verkaufszahlen beeinflussen. In den Jahren 2011 und 2012 war der Monat Januar jeweils der Monat mit den meisten Verkäufen, während 2011 im Juli und 2012 im Semptember die Menge der Bookings am größten war. Allerdings konnten selbst im Januar 2012  die Bookings um 255%(56588) im Vergleich zum Januar 2011 gesteigert werden. 

Im Folgenden werden die Additiven und die Multiplikativen Veränderungen der Zeitreihe zwischen den Monaten angeschaut, die Rote Linie in den Diagrammen signalisiert hierbei ob der absolute Wert größer oder kleiner als im voerherigen Monat ist, abhängig ob der Wert auf dem Diagramm über oder unter der Linie ist. 

In [ ]:
fig = px.line(df_time.reset_index(), x = 'datetime', y = 'Mul', title='Relative Veränderung von Bookings pro Monat')
fig.add_shape(type = 'line', xref='paper', x0=0, x1=12, opacity=0.5, line_color = 'red', 
              y0=1, y1=1)
fig.show()

In [ ]:
fig = px.line(df_time.reset_index(), x = 'datetime', y = 'Add', title='Additive Veränderung von Bookings pro Monat')
fig.add_shape(type = 'line', xref='paper', x0=0, x1=12, opacity=0.5, line_color = 'red', 
              y0=0, y1=0)
fig.show()

In [ ]:
# Ermittlung der Regressionsgerade
from sklearn.linear_model import LinearRegression

X = df_time[['Monat_t']]
y = df_time[['Bookings']]

model = LinearRegression(fit_intercept=True) # Verschiebung auf der y-Achse wird zugelassen
model.fit(X,y)
df_time['Prediction'] = model.predict(X)
print('Regressionsfunktion aus Statistik : y = ax + b')
print(f"Koeffizient a = {round(float(model.coef_), 2)}")
print(f"Koeffizient b = {round(float(model.intercept_), 2)}")

Mittles Sklearn lässt sich die Regressionsgerade für die Bookings ermitteln, dise stellt eine Trendgerade für die Zeitreihe dar. Für die Trendgerade ergibt sich die Gleichung, y = 5760.29 * t + 65330.56, also pro Monat erhöht sich der Trend um 5760.29 Bookings mit einem Anfangswert von 65330.56 Bookings.  

In [ ]:
# Visualierung der Trendgerade und den monatlichen Werten
px.line(df_time.reset_index(), x = 'datetime', y = ['Bookings', 'Prediction'], title = 'Bookings pro Monat und die Trendkomponente der Zeitreihe')

In [ ]:
# Visualierung des Unterschieds zwischen den Trendgeraden und den monatlichen Werten
df_time['Unterschied_Prognose'] = df_time['Bookings'] - df_time['Prediction']
fig = px.area(df_time.reset_index(), x = 'datetime', y = 'Unterschied_Prognose', title='Unterschied zum Trend')
fig.add_shape(type = 'line', xref='paper', x0=0, x1=12, opacity=0.5, line_color = 'red', 
              y0=0, y1=0)

Auf dem Diagramm können die Schwankungen der Zeitreihe um die Trendgerade beobachtet werden, man erkennt hier, dass die Schwankungen periodisch sind, also in manchen Perioden liegt der eigentliche Wert über dem Trend und in manchen Phasen unter dem Trend. 

In [ ]:
# Ermittlung der Saisonalen Einflüsse
df_time['Monat'] = pd.DatetimeIndex(df_time.index).month # Monat wird aus dem Index extrahiert. 

# Gruppierung nach dem Monat liefert die durschnittliche Abweichung pro Monat
df_time_seasonal = df_time.groupby(['Monat'], as_index=False)[['Unterschied_Prognose']].mean()
df_time_seasonal.columns = ['Monat', 'Saisonale_Komponente']

# Die durchschnittliche Abweichung wird dem df hinzugefügt
df_time_sc = pd.merge(df_time.reset_index(), df_time_seasonal, on = 'Monat').sort_values('datetime')
df_time_sc = df_time_sc.set_index('datetime')

# durch Addition des Trends mit den monatlichen Werten ergibt sich ein erster Verlauf
df_time_sc['Seasonal_Prediction'] = df_time_sc['Prediction'] + df_time_sc['Saisonale_Komponente']
df_time_sc

In [ ]:
px.line(df_time_sc.reset_index(), x = 'datetime', y = ['Bookings','Prediction', 'Seasonal_Prediction'],
         title = 'Bookings, Trendgerade und eine erste Vorsage für 2011 und 2012')

## Vorhersage für die zukünftigen Monate
### Werden die Saisonalen Werte zu dem jeweiligen Wert der Trendgerade addiert kann eine erste Vorhersage aufgestellt werden

In [ ]:
# Erstellung eines DF welcher das Jahr 2013 enthält
df_2013_22 = pd.DataFrame(pd.date_range(start = '2013-01-01', end='2013-12-31', freq='M'))
df_2013_22.columns = ['datetime']
df_2013_22['Jahr'] = df_2013_22.datetime.dt.year
df_2013_22['Monat'] = df_2013_22.datetime.dt.month
df_2013_22['Monat_t'] = range(25,37)  

# Bildung der Trendgerade
df_2013_22['Prediction'] = model.predict(df_2013_22[['Monat_t']])

# monatliche Abweichungen werden zum df hinzugefügt und addiert
df_2013_22 = pd.merge(df_2013_22, df_time_seasonal, on='Monat').sort_values('datetime')
df_2013_22['Seasonal_Prediction'] = df_2013_22['Prediction'] + df_2013_22['Saisonale_Komponente']
df_2013_22['datetime'] = pd.to_datetime(df_2013_22['datetime'])
df_2013_22 = df_2013_22.set_index('datetime')
df_2013_22

In [ ]:
# die Prognose für das Jahr 2013 wird an den monatlichen DF angeheftet
df_viz = df_time_sc[['Bookings', 'Seasonal_Prediction', 'Prediction']]
df_viz = pd.concat([df_viz, df_2013_22[['Seasonal_Prediction', 'Prediction']]])
df_viz

In [ ]:
# Änderung der Formatierung
fig = px.line(df_viz.reset_index(), x = 'datetime', y = ['Seasonal_Prediction', 'Prediction', 'Bookings'],
        title='Bookings bis 2013 und Trend/Vorhersage bis Ende 2014')

# fig.update_layout(plot_bgcolor = "#fff", x_axis = 15)
# fig.update_traces(line = {'color' : 'Black', 'width' : 2})
fig

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.ar_model import AutoReg

model1 = ExponentialSmoothing(df_time_sc['Bookings'],
                             freq = 'M',
                             trend = 'add', 
                             seasonal = 'add',
                             seasonal_periods = 12).fit(smoothing_level = 0.9)

model2 = ExponentialSmoothing(df_time_sc['Bookings'],
                             freq = 'M',
                             trend = 'add', 
                             seasonal = 'add',
                             seasonal_periods = 12).fit(smoothing_level = 0.1)

df_time_sc.index = pd.to_datetime(df_time_sc.index)

model_autoreg = AutoReg(df_time_sc['Bookings'],None,trend='ct', seasonal=True, period = 12) # period in DF information
res = model_autoreg.fit()


df_viz['Exponential_Smoothing0.9'] = model1.predict(start = 0, end = 36)
df_viz['Exponential_Smoothing0.1'] = model2.predict(start = 0, end = 36)
df_viz['AutoReg'] = res.predict(start = 0, end = 36)

In [ ]:
fig = px.area(df_viz.loc[ : '2012'], y = 'Bookings',
        title='Bookings bis 2013 und Trend/Vorhersage bis Ende 2014')

fig.update_traces(line = {'color' : 'lightGreen', 'width' : 0.3})

fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['Prediction'], line = {'dash' : 'dot'}, name='Trend')
fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['Seasonal_Prediction'], line = {'color' : 'purple'}, name='Prognose' )
fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['Exponential_Smoothing0.9'], line = {'color' : 'blue'}, name='Exponentielle_Glättung_0.9')
fig.add_scatter(x = df_viz.reset_index()['datetime'], y = df_viz.reset_index()['Exponential_Smoothing0.1'], line = {'color' : '#2E8B57'}, name='Exponentielle_Glättung_0.1')


fig.show()

In [ ]:
# Ermittlung des Modells mit geringster Abweichung
from sklearn.metrics import r2_score

df_met = df_viz.loc[ : '2012']
print(r2_score(df_met['Bookings'], df_met['Seasonal_Prediction'])) # -> am höchsten da es nicht geglättet ist!
print(r2_score(df_met['Bookings'], df_met['Exponential_Smoothing0.9']))
print(r2_score(df_met['Bookings'], df_met['Exponential_Smoothing0.1']))
print(r2_score(df_met['Bookings'], df_met['Exponential_Smoothing0.1']))
print(r2_score(df_met['Bookings'], df_met['AutoReg']))

In [ ]:
print(f"Max von 2013 : {df_viz.loc['2013' : ]['AutoReg'].idxmax()} mit {df_viz.loc['2013' : ]['AutoReg'].max()} Bookings")
print(f"Min von 2013 : {df_viz.loc['2013' : ]['AutoReg'].idxmin()} mit {df_viz.loc['2013' : ]['AutoReg'].min()} Bookings")
print(f"Durchschnitt von 2013 : Average = {df_viz.loc['2013' : ]['AutoReg'].mean()} Bookings")
print(f"Summe von 2013 : Summe = {df_viz.loc['2013' : ]['AutoReg'].sum()} Bookings")
print(f"Die Prozentuale Veränderung gegenüber 2012 : {round((df_viz.loc['2013' : ]['AutoReg'].sum() / df_viz.loc['2012']['Bookings'].sum() -1 ) * 100,2)}%")

Die Bildung einer Prognose für die zukünftigen Jahre ist dabei möglich durch die lineare Fortschreibung sowie mit der exponentiellen Glättung. Man erhält hier je nach Parameter und Methode verschiedene Prognosewerte für 2013 mit der Annahme, dass die bisherige Entwicklung in der Zukunft weiterläuft. Ein Modell sieht dabei für 2013 den Januar mit 168362 Buchungen als schwächsten Monat und den Juni mit 277493 Buchungen als stärksten Monat. Des Weiteren werden 241019 Buchungen im Schnitt vorhergesagt und für das gesamte Jahr 2892233 Buchungen. Dies entspricht einem Wachstum gegenüber 2012 von 40.53%. 

## Tägliche Vorhersage mittels der Regressionsanalyse

Eine Fragestellung, welche sich eher auf das operationale Tagesgeschäft bezieht und dabei die einzelnen Elemente unserer Analyse vereint, ist die Vorhersage der täglichen Nutzung der Scooter und dessen Einflussfaktoren.

In [ ]:
# df_day['weather_name']
px.scatter(df_day, x = 'atemp', y = 'Bookings', color = 'year')

Auf dem Scatterplot kann bereits erkannt werden,dass ein Zusammenhang zwischen der gefühlten Temperatur und den Bookings vorliegt. Da die Bookings über die Jahre hinweg zugenommen haben, wird der Zusammenhang noch größer wenn man ihn abhängig vom Jahr betrachtet

In [ ]:
print(f"Korrelation zwischen Bookings und atemp : {np.corrcoef(df_day['Bookings'], df_day['atemp'])[0,1]}")
mask = (df_day.year == 2011) 
print(f"Korrelation zwischen Bookings und atemp im Jahr 2011 : {np.corrcoef(df_day[mask]['Bookings'], df_day[mask]['atemp'])[0,1]}")
mask = df_day.year == 2012
print(f"Korrelation zwischen Bookings und atemp im Jahr 2012 : {np.corrcoef(df_day[mask]['Bookings'], df_day[mask]['atemp'])[0,1]}")



Wird die allgemeine Steigerung der Bookings zwischen 2019 und 2020 beachtet, also dass im Durchschnitt täglich 65.8% mehr gefahren wurde, so ergibt sich eine Korrelation zwischen „atemp“ und „Bookings“ für das Jahr 2011 von 0.79 und für 2012 ergibt sich 0.72. 


In [ ]:

mask = df_day['year'] == 2011
px.scatter(df_day[mask], x = 'atemp', y = 'Bookings', color = 'month')


Des Weiteren muss auch die monatliche Steigerung der Bookings beachtet werden, da es einen allgemeinen Trend über das Jahr gibt. Bei Betrachtung des Scatterplots kann man erkennen, dass spätere Monate im Jahr oftmals bei einer selben gefühlten Temperatur eine höhere Zahl an Bookings haben. Betrachtet man beispielsweise den Januar und den Dezember, hat der Dezember oft bei gleicher Temperatur, über viele Werte hinweg, mehr Bookings. 


In [ ]:

df_day.groupby(['weather_clear, few clouds'])['Bookings'].mean()


In [ ]:
df_day.groupby(['weather_cloudy, mist'])['Bookings'].mean()


In [ ]:
df_day.groupby(['weather_light snow or rain or thunderstorm'])['Bookings'].mean()


In [ ]:

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

x = df_day[['atemp', 'year', 'weather_clear, few clouds',    
            'weather_light snow or rain or thunderstorm','month']]
y = df_day[['Bookings']]

scaler = MinMaxScaler()
scaler.fit(x, y)
x = scaler.transform(x)

# poly = PolynomialFeatures()
# x = poly.fit_transform(x)

X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.8, random_state=42)

model = Lasso()
model.fit(x,y)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))
model.coef_


In [ ]:

df_coef = pd.DataFrame(model.coef_.reshape(1,-1), columns=['atemp', 'Jahr', 'weather_clear, few clouds', 
            'weather_light snow or rain or thunderstorm', 'Monat'] )
df_coef


In [ ]:

df_day['Predict'] = model.predict(x)
px.line(df_day, x = 'datetime', y = ['Bookings', 'Predict'])



## Modell mit besserem r2 Score



Benutzt man bei dem Modell mehr Features wie Beispielsweise Workingday, Windspeed und Humidity wird das Modell zwar komplexer, allerdings wird hier die Annahme getroffen, dass das Modell durch die Erhöhung der Informationen auch genauere Vorhersgaen treffen kann. Außerdem kann mit der Wetterspalte der r2_score erhöht werden, dieser war zuvor Dummy-codiert wobei immer der Wert welcher an dem Tag am meisten vorgekommen ist eine 1 hatte und die anderen Werte eine 0. Mit der Verwendung von einer prozentualen Skalierung bekommt das Modell so mehr Informationen. Das Wetter am 1. Januar wäre zur Interpretation also zu 31.17% klar gewesen, so 61.45% bewölkt und so 7.36% hat es geschneit.


In [ ]:

df_dummy = pd.get_dummies(df[['datetime', 'weather_name']], columns=['weather_name'])
df_dummy = df_dummy.groupby(['datetime']).mean()
df_dummy.index = pd.to_datetime(df_dummy.index)
df_dummy = df_dummy.resample('d').mean()
df_dummy = df_dummy.reset_index()
df_dummy.columns = ['datetime', 'weather_clear', 'weather_cloudy', 'weather_heavy', 'weather_snow']
df_dummy.head()


In [ ]:

df_reg = pd.merge(df_day, df_dummy, on = 'datetime')
df_reg.head()


In [ ]:

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

col = ['atemp', 'humidity','windspeed', 'year',
       'workingday', 'weather_clear', 'weather_cloudy', 'weather_heavy', 'weather_snow', 'month']
xdf_wm = df_reg[col]
xdf_wm = pd.get_dummies(xdf_wm, columns=['month', 'year'])
ydf_wm = df_reg[['Bookings']]

# Linear Regression
X_train, X_test, y_train, y_test = train_test_split(xdf_wm, ydf_wm, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))


In [ ]:

df_day['Prediction'] = model.predict(xdf_wm)
px.line(df_day, x = 'Datum', y = ['Bookings', 'Prediction'])



Das neue Modell mit einer erhöhten Anzahl an Features erreicht auf den Trainingsdaten einen Wert von 0.8945 und auf den Testdaten von 0.8893, die Features erhöhen also das Modell. Den besten R2_Score erziehlt das Modell wenn Interaktionen zwischen den Features erlaubt werden. Dies klappt unter anderem gut weil wir einen Spezialfall der Interaktion zwischen qualitativen Features welche Dummy-Kodiert sind und quantitativen Features haben. Das Modell erreicht einen R2_Score von 0.9518 auf den Trainingsdaten und 0.941 auf den Testdaten. Werden in einem Liniendiagramm die eigentlichen Werte mit den Predictions des Modells verglichen, kann erkannt werden, dass das Modell zwar nicht die gesamte Varianz der Daten erklären kann, allerdings ist die Tendenz des Modells nahe an den eigentlichen Werten und das sogar bei Ausreiserwerten. Gerade stärkere Schwankunden kann das Modell so erkennen. 


In [ ]:

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

col = ['atemp', 'humidity','windspeed', 'year',
       'workingday', 'weather_clear', 'weather_cloudy', 'weather_heavy', 'weather_snow', 'month']
xdf_wm = df_reg[col]
xdf_wm = pd.get_dummies(xdf_wm, columns=['month', 'year'])
ydf_wm = df_reg[['Bookings']]

# Interaction Term
poly = PolynomialFeatures(interaction_only=True)
xdf_wm = poly.fit_transform(xdf_wm)

# Linear Regression
X_train, X_test, y_train, y_test = train_test_split(xdf_wm, ydf_wm, test_size=0.33, random_state=42)

model = Ridge()
model.fit(X_train, y_train)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))


In [ ]:

df_day['Prediction'] = model.predict(xdf_wm)
px.line(df_day, x = 'Datum', y = ['Bookings', 'Prediction'], title = 'Vergleich zwischen Modell und eigentlichen Werten')



# Regression auf Stunde



Da nun ein Modell besteht welches die täglichen Werte prognostiziert steht natürlich die Vermutung im Raum ob es auch ein Modell gibt welches die stündlichen Werte vorhersagt. Gerade da eine Abhängigkeit besteht zwischen der Tageszeit und den Bookings zu dieser Tageszeit, welche durch Bedingungen wie, ob es sich um einen Arbeitstag handelt, verstärkt werden kann.(Verweis auf vorherige Analysen.) Da die Werte zu den jeweiligen Tageszeiten stündlichen Schwankungen unterliegen werden diese durch die Bildung von Dummy-Variablen beachtet. Das Modell erreicht hier einen Score von 0.9 auf den Trainingsdaten und 0.8878 auf den Testdaten. Bei genauerer Betrachtung des Modells fällt allerdings auf, dass oftmals in den frühen Morgestunden in denen die niedrigste tägliche Nutzung vorliegt, das Modell negative Werte prognostiziert. Damit diser Effekt verhindert werden kann, werden alle Prognosen welche kleiner als 0 sind als 0 angenommen, also das zu dieser Stunde niemand fährt. Nichtsdestotrotz, wird in den Daten nach einem spezifischen Kalenderwoche gefiltert(Beispiel KW:32, 2011), sieht man, dass das Modell die stündlichen Schwankungen erkennt. Beispielsweise erkennt das Modell die Entwicklung an einem Arbeitstag und das morgentliche und abendliche Hoch in der Nutzung. Auch das Wochenende und die veränderte stündliche Nutzung am Wochenende kann hier erkannt werden. 

In [ ]:
df_dummy = pd.get_dummies(df[['datetime', 'weather_name']], columns=['weather_name'])
df_dummy = df_dummy.groupby(['datetime']).mean()
df_dummy.index = pd.to_datetime(df_dummy.index)
df_dummy = df_dummy.resample('h').mean()
df_dummy = df_dummy.reset_index()
df_dummy.head()


In [ ]:
df_hourlyy = pd.merge(df_hourly, df_dummy, on = 'datetime')


In [ ]:

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

col = ['atemp', 'humidity','windspeed', 'year','workingday', 
       'month', 'hour', 'weather_name_clear, few clouds', 'weather_name_cloudy, mist', 'weather_name_heavy rain or thunderstorm or snow or ice pallets', 'weather_name_light snow or rain or thunderstorm']
xdf_wm = df_hourlyy[col]
ydf_wm = df_hourlyy[['Bookings']]
xdf_wm = pd.get_dummies(xdf_wm, columns=['month', 'hour'])

# Interaction Term
poly = PolynomialFeatures(interaction_only=True)
xdf_wm = poly.fit_transform(xdf_wm)

# Linear Regression
X_train, X_test, y_train, y_test = train_test_split(xdf_wm, ydf_wm, test_size=0.35, random_state=42)

model = Ridge()
model.fit(X_train, y_train)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))


In [ ]:

df_hourlyy['Prediction'] = model.predict(xdf_wm).round()
df_hourlyy.head()
def get_null(val):
    if val < 0:
        return 0
    else:
        return val
df_hourlyy['Prediction'] = df_hourlyy['Prediction'].apply(get_null)
print(r2_score(df_hourlyy['Bookings'], df_hourlyy['Prediction']))


In [ ]:

mask = (df_hourlyy['year'] == 2011) & (df_hourlyy['month'] == 8)
px.line(df_hourlyy[mask], x = 'datetime', y = ['Bookings', 'Prediciton'])


In [ ]:

mask = (df_hourlyy['year'] == 2011) & (df_hourlyy['month'] == 8) & (df_hourlyy['week_of_year'] == 32)
fig = px.line(df_hourlyy[mask].reset_index(), x = 'datetime', y = 'Bookings',
        title = 'Vergleich zwischen den stündlichen Bookings und der Prognose in der Kalenderwoche 32 im Jahr 2011')
fig.add_scatter(x = df_hourlyy[mask].reset_index()['datetime'], y = df_hourlyy[mask]['Prediction'], line = {'dash' : 'dot'}, name='Trend')


In [ ]:

## Resampling auf Tagesebene

df_hourlyy = df_hourlyy.set_index('datetime')
df_day_pred = df_hourlyy.resample('d').sum()
df_day_pred.head()


In [ ]:

px.line(df_day_pred.reset_index(), x = 'datetime', y = ['Bookings', 'Prediciton'])


In [ ]:

r2_score(df_day_pred['Bookings'], df_day_pred['Prediction'])


In [ ]:
col = ['atemp', 'humidity','windspeed', 'year', 'month', 'workingday']
xdf_wm = df_day[col]
ydf_wm = df_day[['Bookings']]
# Interaction Term
poly = PolynomialFeatures()
xdf_wm = poly.fit_transform(xdf_wm)

X_train, X_test, y_train, y_test = train_test_split(xdf_wm, ydf_wm, test_size=0.2, random_state=42)

model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
print(r2_score(y_train, model.predict(X_train)))
print(r2_score(y_test, model.predict(X_test)))


In [ ]:

df_dummy = pd.get_dummies(df[['datetime', 'weather']])
df_dummy['Datum'] = df_dummy.datetime.dt.date
df_dummy = df_dummy.groupby(['Datum']).mean()
df_dummy = df_dummy.reset_index()
df_dummy